Preamble

In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("eric_jonsson_p1")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


In [2]:
spark_context = spark_session.sparkContext

Questions A.1

In [3]:
# 1.1

textfile_eng = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.en")
textfile_swe = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.sv")
print(textfile_swe.first())

Återupptagande av sessionen


In [4]:
textfile_swe.take(10)

['Återupptagande av sessionen',
 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december. Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester.',
 'Som ni kunnat konstatera ägde "den stora år 2000-buggen" aldrig rum. Däremot har invånarna i ett antal av våra medlemsländer drabbats av naturkatastrofer som verkligen varit förskräckliga.',
 'Ni har begärt en debatt i ämnet under sammanträdesperiodens kommande dagar.',
 'Till dess vill jag att vi, som ett antal kolleger begärt, håller en tyst minut för offren för bl.a. stormarna i de länder i Europeiska unionen som drabbats.',
 'Jag ber er resa er för en tyst minut.',
 '(Parlamentet höll en tyst minut.)',
 'Fru talman! Det gäller en ordningsfråga.',
 'Ni känner till från media att det skett en rad bombexplosioner och mord i Sri Lanka.',
 'En av de personer som mycket nyligen mördades i Sri Lanka var Kumar Ponnambalam, som besökte Europaparlamentet för bara några månader se

In [5]:
# Split each line into lists

lines_eng = textfile_eng.map(lambda line: line.split('\n'))
lines_swe = textfile_swe.map(lambda line: line.split('\n'))

In [6]:
lines_swe.take(10)

[['Återupptagande av sessionen'],
 ['Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december. Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester.'],
 ['Som ni kunnat konstatera ägde "den stora år 2000-buggen" aldrig rum. Däremot har invånarna i ett antal av våra medlemsländer drabbats av naturkatastrofer som verkligen varit förskräckliga.'],
 ['Ni har begärt en debatt i ämnet under sammanträdesperiodens kommande dagar.'],
 ['Till dess vill jag att vi, som ett antal kolleger begärt, håller en tyst minut för offren för bl.a. stormarna i de länder i Europeiska unionen som drabbats.'],
 ['Jag ber er resa er för en tyst minut.'],
 ['(Parlamentet höll en tyst minut.)'],
 ['Fru talman! Det gäller en ordningsfråga.'],
 ['Ni känner till från media att det skett en rad bombexplosioner och mord i Sri Lanka.'],
 ['En av de personer som mycket nyligen mördades i Sri Lanka var Kumar Ponnambalam, som besökte Europaparlamentet för ba

In [7]:
# Each line is separated into lists, count the number of lists to get number of lines

lineno_eng = lines_eng.map(lambda s: len(s))
total_eng = lineno_eng.reduce(lambda a, b: a+b)
lineno_swe = lines_swe.map(lambda s: len(s))
total_swe = lineno_swe.reduce(lambda a, b: a+b)

print('Total Number of Lines: \nEnglish Transcript: ', total_eng, '\nSwedish Transcript: ', total_swe)

Total Number of Lines: 
English Transcript:  1862234 
Swedish Transcript:  1862234


In [8]:
# 1.4
partitions_eng = textfile_eng.getNumPartitions()
partitions_swe = textfile_swe.getNumPartitions()

print('Partitions: \nEnglish Transcript: ', partitions_eng, '\nSwedish Transcript: ', partitions_swe )

Partitions: 
English Transcript:  2 
Swedish Transcript:  3


Questions A.2

In [9]:
# 2.1

# To lowercase
lower_eng = textfile_eng.map(lambda x: x.lower())
lower_swe = textfile_swe.map(lambda x: x.lower())

In [10]:
print("English: \n")
print(lower_eng.take(10))
print('\n')
print("Swedish: \n")
print(lower_swe.take(10))

English: 

['resumption of the session', 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'you have requested a debate on this subject in the course of the next few days, during this part-session.', "in the meantime, i should like to observe a minute' s silence, as a number of members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the european union.", "please rise, then, for this minute' s silence.", "(the house rose and observed a minute' s silence)", 'madam president, on a point of order.', 'you will be aware from the press and television that there have

In [11]:
# Flatten and split on space
lower_split_eng = lower_eng.map(lambda x: x.split(' '))
lower_split_swe = lower_swe.map(lambda x: x.split(' '))

In [12]:
print("English: ", lower_split_eng.take(10))
print("Swedish: ", lower_split_swe.take(10))

English:  [['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf

In [15]:
print('Total Number of Lines: \nEnglish Transcript: ', lower_split_eng.count(), '\nSwedish Transcript: ', lower_split_swe.count())

Total Number of Lines: 
English Transcript:  1862234 
Swedish Transcript:  1862234


Questions 1.A.3

In [79]:
#A.3.1

from operator import add
def wordcountMap(line):
    for word in line:
        return (word, 1)

wordcount_mapstage_eng = lower_split_eng.map(wordcountMap)
wordcount_mapstage_swe = lower_split_swe.map(wordcountMap)
print(all_words_and_count.take(10))

[('resumption', 1), ('i', 1), ('although,', 1), ('you', 1), ('in', 1), ('please', 1), ('(the', 1), ('madam', 1), ('you', 1), ('one', 1)]


In [80]:
wordcount_eng = wordcount_mapstage_eng.reduceByKey(add)
wordcount_eng.take(10)

wordcount_swe = wordcount_mapstage_swe.reduceByKey(add)
wordcount_swe.take(10)

[('som', 17218),
 ('fru', 12122),
 ('ja,', 1319),
 ('det', 198923),
 ('under', 8440),
 ('tack,', 1302),
 ('nu', 6858),
 ('de', 34602),
 ('nästa', 5430),
 ('eftersom', 6159)]

In [81]:
wordcount_eng_sorted = wordcount_eng.sortBy(ascending = False, keyfunc = lambda el: el[1]).take(10)
print("10 Most Common Words: English")
print(wordcount_eng_sorted, '\n')

wordcount_swe_sorted = wordcount_swe.sortBy(ascending = False, keyfunc = lambda el: el[1]).take(10)
print("10 Most Common Words: Swedish")
print(wordcount_swe_sorted)


10 Most Common Words: English
[('the', 235844), ('i', 201870), ('we', 129603), ('it', 105177), ('in', 91596), ('this', 82526), ('mr', 56025), ('that', 36863), ('as', 33289), ('\xa0\xa0', 30200)] 

10 Most Common Words: Swedish
[('jag', 207086), ('det', 198923), ('vi', 133076), ('i', 66383), ('detta', 51184), ('för', 48686), ('herr', 46659), ('den', 43543), ('de', 34602), ('men', 30438)]
